In [1]:
# Import libraries and set desired options
%matplotlib inline
from matplotlib import pyplot as plt

import pickle
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

Notebook by Yuri Kashnitsky, edited by Ivan Komarov. 

In this competition we are going to analyze a sequence of websites visited by a person to predict whether this person is Alice or not. The metric of evaluation is [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). 

###  Data Downloading and Transformation
First, read the training and test sets. 

In [2]:
times = ['time'+str(i) for i in range(1,11)]
times

['time1',
 'time2',
 'time3',
 'time4',
 'time5',
 'time6',
 'time7',
 'time8',
 'time9',
 'time10']

In [3]:
# Read the training and test data sets and parse dates
train_df = pd.read_csv('train.csv',
                       index_col='session_id', parse_dates=times)

test_df = pd.read_csv('test.csv',
                      index_col='session_id', parse_dates=['time1'])

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
27554,41475,2013-11-15 07:39:35,6725.0,2013-11-15 07:39:35,6725.0,2013-11-15 07:39:36,41475.0,2013-11-15 07:39:36,41476.0,2013-11-15 07:39:40,...,2013-11-15 07:39:41,6725.0,2013-11-15 07:42:50,41475.0,2013-11-15 07:42:50,41476.0,2013-11-15 07:42:50,6725.0,2013-11-15 07:44:25,0
81350,41476,2013-11-15 07:44:25,41475.0,2013-11-15 07:44:25,41476.0,2013-11-15 07:57:45,6725.0,2013-11-15 07:57:45,41475.0,2013-11-15 07:57:45,...,2013-11-15 07:57:46,41476.0,2013-11-15 07:57:47,6725.0,2013-11-15 07:57:49,41475.0,2013-11-15 07:57:49,41476.0,2013-11-15 07:57:49,0
234665,4802,2013-11-15 07:52:17,23.0,2013-11-15 07:52:18,4803.0,2013-11-15 07:52:19,38.0,2013-11-15 07:52:19,38.0,2013-11-15 07:52:20,...,2013-11-15 07:52:20,4804.0,2013-11-15 07:52:23,21.0,2013-11-15 07:52:26,23.0,2013-11-15 07:52:26,22.0,2013-11-15 07:52:28,0
97610,23,2013-11-15 07:52:28,23.0,2013-11-15 07:52:29,22.0,2013-11-15 07:52:37,21.0,2013-11-15 07:52:37,63.0,2013-11-15 07:55:10,...,2013-11-15 07:55:10,784.0,2013-11-15 07:55:56,4804.0,2013-11-15 07:57:50,4804.0,2013-11-15 08:01:18,784.0,2013-11-15 08:01:26,0
161358,41476,2013-11-15 07:57:50,41476.0,2013-11-15 07:57:51,6725.0,2013-11-15 07:59:34,41475.0,2013-11-15 07:59:34,41476.0,2013-11-15 07:59:34,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 27554 to 11690
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   site1   253561 non-null  int64         
 1   time1   253561 non-null  datetime64[ns]
 2   site2   250098 non-null  float64       
 3   time2   250098 non-null  datetime64[ns]
 4   site3   246919 non-null  float64       
 5   time3   246919 non-null  datetime64[ns]
 6   site4   244321 non-null  float64       
 7   time4   244321 non-null  datetime64[ns]
 8   site5   241829 non-null  float64       
 9   time5   241829 non-null  datetime64[ns]
 10  site6   239495 non-null  float64       
 11  time6   239495 non-null  datetime64[ns]
 12  site7   237297 non-null  float64       
 13  time7   237297 non-null  datetime64[ns]
 14  site8   235224 non-null  float64       
 15  time8   235224 non-null  datetime64[ns]
 16  site9   233084 non-null  float64       
 17  time9   233084 non-null  d

In [18]:
# Задание 1
# Какой процент сессий Элис из всех сессий?

# количество сессий Алисы = 2297
# len(train_df[train_df['target'] == 1])
# общее количество сессий = 253561
# len(train_df[train_df['target'] == 1])

# процент сессий Алисы:

len(train_df[train_df['target'] == 1]) / len(train_df) * 100

0.905896411514389

In [11]:
# Задание 2
# В какой день недели Элис появилась в первый раз?

# Отсеиваем сессии в которых была Алиса

session_Alica = train_df[train_df['target'] == 1]

# выбираем первую сессию

session_fist = session_Alica.head(1)
session_Alica_time1 = session_Alica["time1"]
session_Alica_time1 = sorted(session_Alica_time1)

# Определяем дату и время первого посещения
# session_fist["time1"]
# 2013-11-15 12:18:58

import datetime
# Пример на сегодняшний день (отсчёт с 0)
# datetime.datetime.today().weekday()

# Дата
data_1 = session_fist["time1"].dt.dayofweek
# data_1 = session_Alica_time1[0]
# 2013-11-15 12:18:58

data_1

# 4 - пятница

session_id
79635   2013-11-15 12:18:58
Name: time1, dtype: datetime64[ns]

In [78]:
# Задание 3
# Какие года присутствуют в обучающей выборке?

# Пока сделаю грубый поиск + по всем столбцам
# пример извлечения даты:
# train_df['time1'][:].dt.year

# !!Поиск!!

# len(train_df[train_df['time1'].dt.year == 2013])
# 74675 - значит есть 2013

# len(train_df[train_df['time1'].dt.year == 2014])
# 178886 - значит есть 2014

len(train_df[train_df['time10'].dt.year == 2011])
# time1:0 - значит нет 2011
# time2:0 - значит нет 2011
# time3:0 - значит нет 2011
# time4:0 - значит нет 2011
# time5:0 - значит нет 2011
# time6:0 - значит нет 2011
# time7:0 - значит нет 2011
# time8:0 - значит нет 2011
# time9:0 - значит нет 2011
# time10:0 - значит нет 2011

0

In [ ]:
# Задание 4
# Какой сайт в обучающей выборке чаще всего посещает первым Элис и обычный пользователь?

# Перемещаюсь ниже, так как нужен файл "site_dict"


The training data set contains the following features:

- **site1** – ID of the first visited website in the session
- **time1** – visiting time for the first website in the session
- ...
- **site10** – ID of the tenth visited website in the session
- **time10** – visiting time for the tenth website in the session
- **target** – target variable, equals 1 for Alice's sessions, and 0 otherwise
    
**User sessions end either if a user has visited ten websites or if a session has lasted over thirty minutes.**

There are some empty values in the table, it means that some sessions contain less than ten websites. Replace empty values with 0 and change columns types to integer. Also load the websites dictionary and check how it looks:

In [79]:
# Change site1, ..., site10 columns type to integer and fill NA-values with zeros
sites = ['site'+str(i) for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# Load websites dictionary
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)
    
# r before a string means "raw", i.e. take the string as it comes,
# e.g. as a file path without interpreting special symbols like \n

print('Websites total:', len(site_dict))



Websites total: 48371


In [128]:
# Задание 4
# Какой сайт в обучающей выборке чаще всего посещает первым Элис и обычный пользователь?

# type(site_dict)

# Пример извлечения данных
# site_dict['s.youtube.com']

# Переменные уже есть, но дублирую их
session_Alica = train_df[train_df['target'] == 1]
session_Ordinary = train_df[train_df['target'] == 0]
session_Alica_site1 = session_Alica["site1"]
session_Ordinary_site1 = session_Ordinary["site1"]


len(session_Alica_site1)

# len(set(session_Alica_site1))
# 448
# len(session_Alica_site1)
# 2297

# счётчик использования
from collections import Counter
occurrences_Alica = Counter(session_Alica_site1)
occurrences_Alica_MostCommon = occurrences_Alica.most_common()
#(80, 142), s.youtube.com
#(77, 141),
#(76, 130),

occurrences_Ordinary = Counter(session_Ordinary_site1)
occurrences_Ordinary_MostCommon = occurrences_Ordinary.most_common()
#(21, 13794) www.google.fr
#(23, 9193),
#(782, 7769),

#site_dict['rr.office.microsoft.com']

718

In [187]:
# Задание 5
# Какой сайт в обучающей выборке является вторым по популярности в 2014 году и сколько раз его посещали в 2014 году?

# Websites total: 48371

train_df_2014 = train_df[train_df['time1'].dt.year == 2014]

site = ['site'+str(i) for i in range(1,11)]

# список для задания количества - первый столбец, номер сайта, второй - количество
site_count = [[0 for i in range(1,3)] for j in range(1,48371+2)]
# site_count[0][1]

for i in range(1, 48371+1):
    site_count[i][0] = i
    site_count[i][1] = 0

# Функция не оптимизирована, нужно добавит как минимум счётчик по 1к, чтобы отслеживать
for site_one in site:
    site_Analiz = train_df_2014[site_one]
    site_Analiz = site_Analiz[site_Analiz !=0]
    for i in range(1, 48371+1):
        site_count[i][1] += len(site_Analiz[site_Analiz == i])
        
site_count

[[0, 0],
 [1, 1899],
 [2, 217],
 [3, 12216],
 [4, 1193],
 [5, 689],
 [6, 2035],
 [7, 1974],
 [8, 2961],
 [9, 40],
 [10, 980],
 [11, 2339],
 [12, 1370],
 [13, 282],
 [14, 5803],
 [15, 432],
 [16, 372],
 [17, 575],
 [18, 1088],
 [19, 65],
 [20, 749],
 [21, 86090],
 [22, 39681],
 [23, 56333],
 [24, 0],
 [25, 13],
 [26, 3054],
 [27, 2330],
 [28, 313],
 [29, 39273],
 [30, 18070],
 [31, 3649],
 [32, 6781],
 [33, 14651],
 [34, 416],
 [35, 16729],
 [36, 1799],
 [37, 18227],
 [38, 9568],
 [39, 16787],
 [40, 19],
 [41, 269],
 [42, 294],
 [43, 0],
 [44, 0],
 [45, 1157],
 [46, 1835],
 [47, 492],
 [48, 5353],
 [49, 4041],
 [50, 4139],
 [51, 749],
 [52, 22715],
 [53, 4615],
 [54, 552],
 [55, 17431],
 [56, 9228],
 [57, 1630],
 [58, 551],
 [59, 450],
 [60, 2],
 [61, 33],
 [62, 2],
 [63, 6378],
 [64, 115],
 [65, 606],
 [66, 3896],
 [67, 1760],
 [68, 1044],
 [69, 1580],
 [70, 1896],
 [71, 2082],
 [72, 16],
 [73, 754],
 [74, 30],
 [75, 7179],
 [76, 14265],
 [77, 11521],
 [78, 2933],
 [79, 293],
 [80, 190

In [196]:
# Задание 5 продолжение

# осталось отсортировать
def custom_key(site_count):
    return site_count[1]


site_count.sort(reverse=True, key=custom_key)

# [21, 86090],
# [782, 77039],
# [23, 56333],
# [780, 40610],

# site_dict['apis.google.com'] = 22
# site_dict['blast.ncbi.nlm.nih.gov']  = 780
# site_dict['annotathon.org']  = 782
# site_dict['www.google.fr']  = 21
# site_dict['www.google.com']  = 23

site_dict['www.google.com']

23

In [199]:
# Задание 6
# Сколько всего сессий обучающей выборке содержат в себе меньше чем 10 сайтов?

train_site10_0 = train_df[train_df['site10'] == 0]
# len(train_site10_0) = 22509

22509

In [ ]:
# Задание 7
# Выберите верные утверждения для обучающей выборки:

# Медианная длительность посещения первой страницы в среднем больше, чем у Элис
# Элис наиболее активна в марте и неактивна летом
# Лето является самым малоактивным временем года вообще
# В среднем Элис проводит меньше времени на первой странице, чем другие пользователи

In [100]:
# Задание 7
# Выберите верные утверждения для обучающей выборки:

# Медианная длительность посещения первой страницы в среднем больше, чем у Элис
# Не верное утверждение

session_Alica = train_df[train_df['target'] == 1]

session_Alica_befor = len(session_Alica)
session_Alica = session_Alica[pd.isnull(session_Alica["time2"]) != True]
session_Alica_after = len(session_Alica)
print("session_Alica_befor = ", session_Alica_befor)
print("session_Alica_after = ", session_Alica_after)

session_All = train_df

session_All_befor = len(session_All)
session_All = session_All[pd.isnull(session_All["time2"]) != True]
session_All_after = len(session_All)
print("session_All_befor = ", session_All_befor)
print("session_All_after = ", session_All_after)


del_time1_Alica = session_Alica["time2"] - session_Alica["time1"]
del_time1_All = session_All["time2"] - session_All["time1"]


def custom_key(site_count):
    return del_time1_Alica[1]

sorted_del_time1_Alica = sorted(del_time1_Alica)
# median_Alica = sorted_del_time1_Alica[int(session_Alica_after / 2) - 1 : int(session_Alica_after / 2) + 2]
median_Alica = sorted_del_time1_Alica[int(session_Alica_after / 2) - 1]


sorted_del_time1_All = sorted(del_time1_All)
# median_All = sorted_del_time1_All[int(session_All_after / 2) - 1 : int(session_All_after / 2) + 2]
median_All = sorted_del_time1_All[int(session_All_after / 2)]


# median_Alica = statistics.median(del_time1_Alica)
# median_All = statistics.median(del_time1_All)

print("Alica = ", median_Alica)
print("All = ", median_All)
median_Alica < median_All

session_Alica_befor =  2297
session_Alica_after =  2294
session_All_befor =  253561
session_All_after =  250098
Alica =  0 days 00:00:01
All =  0 days 00:00:01


False

[range(0, 12)]

In [127]:
# Задание 7 (продолжение)
# Выберите верные утверждения для обучающей выборки:

# В среднем Элис проводит меньше времени на первой странице, чем другие пользователи
# Да, в среднем меньше
# Alica =  6.397994768962511
# All =  19.19850218714264

avarage_Alica = 0
for one_value in del_time1_Alica:
    avarage_Alica += one_value.seconds
    
avarage_Alica = avarage_Alica / len(del_time1_Alica)

avarage_All = 0  
for one_value in del_time1_All:
    avarage_All += one_value.seconds
    
avarage_All = avarage_All / len(del_time1_All)


print("Alica = ", avarage_Alica)
print("All = ", avarage_All)

Alica =  6.397994768962511
All =  19.19850218714264


In [135]:
# Задание 7 (продолжение)
# Выберите верные утверждения для обучающей выборки:

# Элис наиболее активна в марте и неактивна летом
# Нет, наиболее активна в декабре, хоть про лето и верно

session_Alica = train_df[train_df['target'] == 1]

for month in range(0,15):
    len_month = len(session_Alica[session_Alica["time1"].dt.month == month])
    print("month = ", month, ", len = ",len_month)

month =  0 , len =  0
month =  1 , len =  88
month =  2 , len =  449
month =  3 , len =  399
month =  4 , len =  305
month =  5 , len =  0
month =  6 , len =  0
month =  7 , len =  0
month =  8 , len =  0
month =  9 , len =  0
month =  10 , len =  0
month =  11 , len =  446
month =  12 , len =  610
month =  13 , len =  0
month =  14 , len =  0


In [136]:
# Задание 7 (продолжение)
# Выберите верные утверждения для обучающей выборки:

# Лето является самым малоактивным временем года вообще
# Да, этих месяцев вообще нет

for month in range(0,15):
    len_month = len(train_df[train_df["time1"].dt.month == month])
    print("month = ", month, ", len = ",len_month)

month =  0 , len =  0
month =  1 , len =  28058
month =  2 , len =  53108
month =  3 , len =  55275
month =  4 , len =  42358
month =  5 , len =  87
month =  6 , len =  0
month =  7 , len =  0
month =  8 , len =  0
month =  9 , len =  0
month =  10 , len =  0
month =  11 , len =  31118
month =  12 , len =  43557
month =  13 , len =  0
month =  14 , len =  0


In [277]:
# Задание 8 (тест на Алисе)
# Чему равное медианное значение количества уникальных сайтов в рамках одной сессии обучающей выборки?

import math
import numpy

session_Alica = train_df[train_df['target'] == 1]
number_sites = numpy.zeros(shape=(len(session_Alica),10))
Unique_sites = numpy.zeros(shape=(len(session_Alica),1))

sites = ['site'+str(i) for i in range(1,11)]
i = 0
for one_site in sites:
    site_x = session_Alica[one_site]
    k = 0
    for site_number in site_x:
        number_sites[k][i] = site_number
        k+=1
        
    i+=1
print(number_sites[0:2])

for i in range(0,len(Unique_sites)):
    set_site = {number_sites[i][0]}
    for ind in range(0,10):
        nonan_site = math.isnan(number_sites[i][ind]) != True
        if nonan_site:
            set_site.add(number_sites[i][ind])
    Unique_sites[i] = len(set_site)

Unique_sites = sorted(Unique_sites)
Unique_sites[int(len(Unique_sites)/2)-1:int(len(Unique_sites)/2)+2]

[[ 270.   63.   21.   21.   22.   22.   21. 7832.   21.   30.]
 [7832.   29. 7832.   37. 7832.   30.   29. 7832.   30.   29.]]


[array([6.]), array([6.]), array([6.])]

In [278]:
# Задание 8
# Чему равное медианное значение количества уникальных сайтов в рамках одной сессии обучающей выборки?

import math
import numpy

number_sites = numpy.zeros(shape=(len(train_df),10))
Unique_sites = numpy.zeros(shape=(len(train_df),1))

sites = ['site'+str(i) for i in range(1,11)]
i = 0
for one_site in sites:
    site_x = train_df[one_site]
    k = 0
    for site_number in site_x:
        number_sites[k][i] = site_number
        k+=1
        
    i+=1
print(number_sites[0:2])

for i in range(0,len(Unique_sites)):
    set_site = {number_sites[i][0]}
    for ind in range(0,10):
        nonan_site = math.isnan(number_sites[i][ind]) != True
        if nonan_site:
            set_site.add(number_sites[i][ind])
    Unique_sites[i] = len(set_site)

Unique_sites = sorted(Unique_sites)
Unique_sites[int(len(Unique_sites)/2)-1:int(len(Unique_sites)/2)+2]

[[41475.  6725.  6725. 41475. 41476. 41476.  6725. 41475. 41476.  6725.]
 [41476. 41475. 41476.  6725. 41475. 41476. 41476.  6725. 41475. 41476.]]


[array([6.]), array([6.]), array([6.])]

In [405]:
# Задание 9
# Чему равное медианное значение времени сессии в обучающей выборке (время посещения последнего сайта примем равным нулю)?

session_All = train_df

session_All_befor = len(session_All)
session_All = session_All[pd.isnull(session_All["time2"]) != True]
session_All_after = len(session_All)

# минимальное время
del_time1_All = numpy.zeros(shape=(len(session_All),1))
del_time1_All_2 = session_All["time2"] - session_All["time1"]
del_time1_All_3 = session_All["time3"] - session_All["time1"]
del_time1_All_4 = session_All["time4"] - session_All["time1"]
del_time1_All_5 = session_All["time5"] - session_All["time1"]
del_time1_All_6 = session_All["time6"] - session_All["time1"]
del_time1_All_7 = session_All["time7"] - session_All["time1"]
del_time1_All_8 = session_All["time8"] - session_All["time1"]
del_time1_All_9 = session_All["time9"] - session_All["time1"]
del_time1_All_10 = session_All["time10"] - session_All["time1"]

#times = ['time'+str(i) for i in range(10,2,-1)]
times = ['time'+str(i) for i in range(1,11)]
# расчёт максимального времени сессии
session_All = session_All[times]

In [412]:
# Задание 9 (продолжение)
# Чему равное медианное значение времени сессии в обучающей выборке (время посещения последнего сайта примем равным нулю)?

index = session_All.index

for ind in index:
    if

Int64Index([ 27554,  81350, 234665,  97610, 161358, 232738, 133946,  55672,
             82466,  72432,
            ...
             41722, 195431, 227055, 201673,  37102,  41843,  36964,  11678,
            153398,  11690],
           dtype='int64', name='session_id', length=250098)

In [6]:
# See what's in the dict
list(site_dict.items())[:3]

[('www.abmecatronique.com', 25075),
 ('groups.live.com', 13997),
 ('majeureliguefootball.wordpress.com', 42436)]

In [7]:
# Size of the sets
print(test_df.shape, train_df.shape)

(82797, 20) (253561, 21)


In [8]:
# What's inside the train
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
27554,41475,2013-11-15 07:39:35,6725,2013-11-15 07:39:35,6725,2013-11-15 07:39:36,41475,2013-11-15 07:39:36,41476,2013-11-15 07:39:40,...,2013-11-15 07:39:41,6725,2013-11-15 07:42:50,41475,2013-11-15 07:42:50,41476,2013-11-15 07:42:50,6725,2013-11-15 07:44:25,0
81350,41476,2013-11-15 07:44:25,41475,2013-11-15 07:44:25,41476,2013-11-15 07:57:45,6725,2013-11-15 07:57:45,41475,2013-11-15 07:57:45,...,2013-11-15 07:57:46,41476,2013-11-15 07:57:47,6725,2013-11-15 07:57:49,41475,2013-11-15 07:57:49,41476,2013-11-15 07:57:49,0
234665,4802,2013-11-15 07:52:17,23,2013-11-15 07:52:18,4803,2013-11-15 07:52:19,38,2013-11-15 07:52:19,38,2013-11-15 07:52:20,...,2013-11-15 07:52:20,4804,2013-11-15 07:52:23,21,2013-11-15 07:52:26,23,2013-11-15 07:52:26,22,2013-11-15 07:52:28,0
97610,23,2013-11-15 07:52:28,23,2013-11-15 07:52:29,22,2013-11-15 07:52:37,21,2013-11-15 07:52:37,63,2013-11-15 07:55:10,...,2013-11-15 07:55:10,784,2013-11-15 07:55:56,4804,2013-11-15 07:57:50,4804,2013-11-15 08:01:18,784,2013-11-15 08:01:26,0
161358,41476,2013-11-15 07:57:50,41476,2013-11-15 07:57:51,6725,2013-11-15 07:59:34,41475,2013-11-15 07:59:34,41476,2013-11-15 07:59:34,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0


For the very basic model, we will use only the visited websites in the session (we will not take into account timestamp features). 

*Alice has her favorite sites, and the more often you see these sites in the session, the higher probability that this is an Alice session, and vice versa.*

Let us prepare the data, we will take only features `site1, site2, ... , site10` from the whole dataframe. Keep in mind that the missing values are replaced with zero. Here is what the first rows of the dataframe look like:

In [9]:
train_df[sites].head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
27554,41475,6725,6725,41475,41476,41476,6725,41475,41476,6725
81350,41476,41475,41476,6725,41475,41476,41476,6725,41475,41476
234665,4802,23,4803,38,38,4804,4804,21,23,22
97610,23,23,22,21,63,66,784,4804,4804,784
161358,41476,41476,6725,41475,41476,0,0,0,0,0


Since IDs of sites have no meaning (does not matter if a site has an ID of 1 or 100), we need to think about how to encode the meaning of "this site in a session means higher probablity that it is an Alice session". 

We will use a technique called ["bag of words plus n-gram model"](https://en.wikipedia.org/wiki/Bag-of-words_model).

We will make a "site-session" matrix analogous to the term-document matrix.

We are not the first, and luckily there is a function CountVectorizer that will implement the above model. Type help(CountVectorizer) to learn about the function. 

We will now initialize a "cv" (CountVectorizer's) instance which we need to train. 

We will use the following parameters:

_ngram range=(1, 3)_ - here we decide that we will use 
1) the name of the site, 
2) two consecutive site names, and 
3) three consecutive site names as features. 
E.g. "google.com" or "google.com vk.com" or "google.com vk.com groups.live.com". 

CountVectorizer will create a large dictionary of 1, 2, and 3-gram strings of sites represented by their numerical IDs. However, this dictionary will be so so large that we may run into trouble with memory or we will just be inefficent chasing phantom combinations.

We will thus limit the dictionary to 50K of the most frequent n-grams:

_max features=50000_

Here is our empty instance:

In [11]:
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)

CountVectorizer accepts "document strings", so let's prepare a string of our "documents" (i.e. sites), divided by space. Since the string will be huge, we will write this string in a text file using pandas:

In [12]:
train_df[sites].fillna(0).to_csv('train_sessions_text.txt', 
                                 sep=' ', index=None, header=None)
test_df[sites].fillna(0).to_csv('test_sessions_text.txt', 
                                sep=' ', index=None, header=None)

Before we start using CountVectorizer, let's see how it works on a sub-set of 5 sessions:

In [13]:
five_sess = pd.read_csv('train_sessions_text.txt', sep=' ', nrows=5, header=None)

In [14]:
five_sess

,0,1,2,3,4,5,6,7,8,9
0,41475,6725,6725,41475,41476,41476,6725,41475,41476,6725
1,41476,41475,41476,6725,41475,41476,41476,6725,41475,41476
2,4802,23,4803,38,38,4804,4804,21,23,22
3,23,23,22,21,63,66,784,4804,4804,784
4,41476,41476,6725,41475,41476,0,0,0,0,0


First of all, let's make an inverse dictionary which gives us a site name for ID.
The direct dictionary came to us like this:

In [15]:
list(site_dict.items())[:3]

[('www.abmecatronique.com', 25075),
 ('groups.live.com', 13997),
 ('majeureliguefootball.wordpress.com', 42436)]

In [16]:
# The inverse dictionary:

new_dict = {}
for key in site_dict:
    new_dict[site_dict[key]] = key

In [17]:
# Let's check what's in it:

list(new_dict.items())[:3]

[(25075, 'www.abmecatronique.com'),
 (13997, 'groups.live.com'),
 (42436, 'majeureliguefootball.wordpress.com')]

In [18]:
# Let's see site names in the five first sessions:

list_sites = []
for row in five_sess.values:
    row_sites = ' '.join([str(i) for i in row if i!=0])
    print(row_sites)
    list_sites.append(row_sites) 

print()
    
list_sites_names = []
for row in five_sess.values:
    row_sites = ' '.join([new_dict[i] for i in row if i!=0])
    print(row_sites)
    list_sites_names.append(row_sites)

41475 6725 6725 41475 41476 41476 6725 41475 41476 6725
41476 41475 41476 6725 41475 41476 41476 6725 41475 41476
4802 23 4803 38 38 4804 4804 21 23 22
23 23 22 21 63 66 784 4804 4804 784
41476 41476 6725 41475 41476

security.debian.org www-fourier.ujf-grenoble.fr www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org www-fourier.ujf-grenoble.fr
backports.debian.org security.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org
cnfg.toolbarservices.com www.google.com utils.delta-search.com ajax.googleapis.com ajax.googleapis.com img.babylon.com img.babylon.com www.google.fr www.google.com apis.google.com
www.google.com www.google.com apis.google.com www.google.fr ieonline.microsoft.com go.microsoft.com javadl-esd-secure.oracle.com img.babylon.com i

Here is what the fit and transform method -- i.e. learn the dictionary and make the matrix -- produces in our "cv":
a sparse matrix. Why sparse? Because nrows * dict_size = usually will not fit in memory 
(obviously, our 5 sessions will fit in memory so that we can look at them)

In [19]:
see_vect = cv.fit_transform(list_sites)

# Matrix dimensions: 5 sessions of 60 elements
see_vect

<5x60 sparse matrix of type '<class 'numpy.int64'>'
	with 88 stored elements in Compressed Sparse Row format>

In [20]:
# Here is the dictionary of sites, 1 to 3-gram words. First 6 elements in the matrix:

cv.get_feature_names()[:6]

['21', '21 23', '21 23 22', '21 63', '21 63 66', '22']

In [21]:
# A version with the site names. Note that security.debian.org has ID of 21.

for i, string in enumerate(cv.get_feature_names()):
    if i < 21:
        print (i+1, end=" ")
        for num in string.split():
            print(new_dict[int(num)], end=" ")
        print()

1 www.google.fr 
2 www.google.fr www.google.com 
3 www.google.fr www.google.com apis.google.com 
4 www.google.fr ieonline.microsoft.com 
5 www.google.fr ieonline.microsoft.com go.microsoft.com 
6 apis.google.com 
7 apis.google.com www.google.fr 
8 apis.google.com www.google.fr ieonline.microsoft.com 
9 www.google.com 
10 www.google.com apis.google.com 
11 www.google.com apis.google.com www.google.fr 
12 www.google.com www.google.com 
13 www.google.com www.google.com apis.google.com 
14 www.google.com utils.delta-search.com 
15 www.google.com utils.delta-search.com ajax.googleapis.com 
16 ajax.googleapis.com 
17 ajax.googleapis.com ajax.googleapis.com 
18 ajax.googleapis.com ajax.googleapis.com img.babylon.com 
19 ajax.googleapis.com img.babylon.com 
20 ajax.googleapis.com img.babylon.com img.babylon.com 
21 security.debian.org 


In [22]:
# Here is the session-site matrix, toarrray() helps us to see a sparse matrix since it is not large.

see_vect.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2,
        1, 1, 1, 1, 3, 0, 0, 1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 2, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3,
        1, 1, 0, 0, 5, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 3, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 

In [23]:
# The first session (row in the matrix) is this:

list_sites_names[0]

'security.debian.org www-fourier.ujf-grenoble.fr www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org www-fourier.ujf-grenoble.fr'

Let's see how the first site of the first session, "security.debian.org", is recorded in the session-site matrix. 
Its ID is 21 which corresponds to 3. It is the number of times this site was seen in the first session.
Indeed, count for yourself in the cell above. 

In [24]:

first_row = see_vect.toarray()[0]

for one, two in zip(range(60),first_row):
    if one < 21:
        print (one+1, two)


1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 3


Let's go back to all sessions.

Fit `CountVectorizer` to train data and transform the train and test data with it.

In [25]:
%%time

with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)

print(X_train.shape, X_test.shape)

# Note very big dimensions of matrices: 253561 * 50000 = 12678050000 elements in train! Only sparse matrices can take it.

(253561, 50000) (82797, 50000)
Wall time: 21.2 s


### Training the first model

So, we have an algorithm and data for it. Let us build our first model, using [logistic regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) implementation from ` Sklearn` with default parameters. We will use the first 90% of the data for training (the training data set is sorted by time) and the remaining 10% for validation. Let's write a simple function that returns the quality of the model and then train our first classifier:

In [26]:
def get_auc_lr_valid(X, y, C=1.0, seed=17, ratio = 0.9):
    
    # Split the data into the training and validation sets
    idx = int(round(X.shape[0] * ratio))
    
    # Classifier training
    lr = LogisticRegression(C=C, random_state=seed, solver='lbfgs', max_iter=500).fit(X[:idx, :], y[:idx])
    
    # Prediction for validation set
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    
    # Calculate the quality
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [27]:
# Our target variable
y_train = train_df['target'].values

In [28]:
%%time
# Calculate metric on the validation set. 90% of train data for training. 10% for validation.

print(get_auc_lr_valid(X_train, y_train))

0.9122581928805027
Wall time: 21 s


In [29]:
# 50% of train data for training:

get_auc_lr_valid(X_train, y_train, ratio=0.5)

0.8225182301089249

In [30]:
# Wow! Big data rules in this task: .82 -> .91

In [31]:
# Function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = range(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [32]:
# Train the model on the whole training data set
# Use random_state=17 for reproducibility
# Parameter C=1 by default, but here we set it explicitly

lr = LogisticRegression(C=1.0, random_state=17, solver='lbfgs', max_iter=500).fit(X_train, y_train)

# Make a prediction for test data set
y_test = lr.predict_proba(X_test)[:, 1]

# Write it to the file which could be submitted
write_to_submission_file(y_test, 'baseline_1.csv')

The first model demonstrated the quality of 0.9235 on the validation set. Let's take it as the first baseline and starting point. 